<a href="https://colab.research.google.com/github/RogerJL/LTU/blob/main/ANN/ANN_Exercise_1_jax_works.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# D7046E Exercise 1 (ANN1)

This exercise has three taks where you will deepen your understanding of how artificial neural networks (ANNs) are implemented and trained. First, you will represent digits on an eight-segment display as vectors and hard-code perceptrons that classifies the digits. The purpose of this task is to better understand the basic computational units in ANNs and how inputs can be represented as feature vectors. Secondly, you will implement and train neural networks using [pytorch](https://pytorch.org/) on the seven-segment display data and the [MNIST](http://yann.lecun.com/exdb/mnist/) dataset which you are familiar with from Exercise 0. Finally, you will implement a neural network including the forward (inference) pass and the backward (learning) pass from scratch using numpy. After completing these steps you will know the central building blocks of ANNs.

## Literature
Before starting with the implementation you should familiarize yourself with two additional chapters in the [deep learning book](https://www.deeplearningbook.org/). This will help you understand the theory behind neural networks and what mathematical formulas are important for the task. The lectures has touched on most of these concepts. Below is a list of recommended sections from the book. If you feel familiar with the contents of these sections, feel free to skip it.

* Chapter 6 - Deep feedforward networks
    - Section 6.0 - Discusses what do we mean by feedfoward networks and terminology such as input layer, output layer and hidden layer.
    - Section 6.2 - Discusses what gradient based learning is and what cost functions are.
    - Section 6.5 - Explains back propagation. Important here are the formulas 6.49 - 6.52.
* Chapter 8 - Optimization for Training Deep Models
    - Section 8.1.3 - Presents differences between batch (deterministic) and mini-batch (stochastic) algorithms.
    
## Libraries

Before starting with the implementations you need to import the following libraries.

In [1]:
import jax
print("JAX devices", jax.devices())

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import matplotlib.pyplot as plt
import numpy
import copy
import itertools


# Download the dataset and create the dataloaders
mnist_train = datasets.MNIST("./", train=True, download=True, transform=transforms.ToTensor())

# Split training dataset
parts = torch.utils.data.random_split(mnist_train, [0.8, 0.2])

mnist_test = datasets.MNIST("./", train=False, download=True, transform=transforms.ToTensor())


JAX devices [gpu(id=0)]


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available()
                      else "cpu")
print("Execution device", device)


Execution device cuda:0


## Task 1

Classification of seven-segment display numbers with perceptrons. A [seven-segment display](https://en.wikipedia.org/wiki/Seven-segment_display) can be used to display digits by turning the different segments (A,B,C,D,E,F, G) on or off. Your task is to design ten different perceptrons (which together make a single-layer neural network with 10 outputs) that recognizes the ten different digits (0,1,2,3, ... ,9) represented by a seven-segment display. The input to each perceptron will be a seven-dimensional vector {A,B,C,D,E,F,G} where A is 1 if segment A is turned on and 0 otherwise (and the same for all the other segments).

![Seven Segment Display](https://upload.wikimedia.org/wikipedia/commons/thumb/e/ed/7_Segment_Display_with_Labeled_Segments.svg/225px-7_Segment_Display_with_Labeled_Segments.svg.png)

This means that for each digit you should create a vector representing the digit, as well as a perceptron that selectively gives a high output for that particular digit and a low output for other digits. For each digit (0 to 9) the corresponding perceptron shoud have the greatest output value, indicating that it can correctly classify the corresponding digit/vector and thereby distinguish that digit from other digits.

For example, the digit 2 corresponds to the vector {1,1,0,1,1,0,1}. If the perceptron at index 2 (the third pereptron since we have a perceptron for 0 as well) gives the greatest output for this vector then this perceptron functions as desired. Similarly, other perceptrons should give the highest output for their corresponding vector.

For example, the digit zero is represented as segments A-F being on and segment G being off. The zero perceptron should reasonably have positive activation for A-F, say at a value of 1, and negative activation for G, say -1.

Why negative?

If we instead set the activation for G to 0, then there is no distinction between the digits zero and eight. Thus, our perceptron for zero would have the same output regardless of whether the input is zero or eight. You can address this with biases, so that the output for our eight-perceptron is higher than the zero-perceptron for input eight, and vice versa, but it makes balancing weights and biases much more difficult.

A good starting point is to consider which activations should be positive or negative, implement this, then check the output for all perceptrons for each digit and then contemplate the bias.


### Task 1.1

In this task you should use numpy rather than PyTorch to implement and understand the elementary arithmetic operations involved. After completing this exercise you should understand how an artificial neural network unit (like the perceptron) produces one scalar output from multiple input values, and how the trainable weights and biases determine the relation between input and output values of a neural network unit and layer.

Complete the input vectors, the weight vectors, and biases. Then update the prediction calculation (forward pass) to include bias in the calculation.

In [3]:
## First we need to define all the vectors corresponding to the various digits and add them to a list for easy access
# Please finish the list of digit vectors asdasdas
x = [
    numpy.array([1,1,1,1,1,1,0]), # 0
    numpy.array([0,1,1,0,0,0,0]), # 1
    numpy.array([1,1,0,1,1,0,1]), # 2
    numpy.array([1,1,1,1,0,0,1]), # 3
    numpy.array([0,1,1,0,0,1,1]), # 4
    numpy.array([1,0,1,1,0,1,1]), # 5
    numpy.array([1,0,1,1,1,1,1]), # 6
    numpy.array([1,1,1,0,0,0,0]), # 7
    numpy.array([1,1,1,1,1,1,1]), # 8
    numpy.array([1,1,1,1,0,1,1]), # 9
]

# TEACHER SOLUTION
x = [
    numpy.array([1,1,1,1,1,1,0]), # 0
    numpy.array([0,1,1,0,0,0,0]), # 1
    numpy.array([1,1,0,1,1,0,1]), # 2
    numpy.array([1,1,1,1,0,0,1]), # 3
    numpy.array([0,1,1,0,0,1,1]), # 4
    numpy.array([1,0,1,1,0,1,1]), # 5
    numpy.array([1,0,1,1,1,1,1]), # 6
    numpy.array([1,1,1,0,0,0,0]), # 7
    numpy.array([1,1,1,1,1,1,1]), # 8
    numpy.array([1,1,1,1,0,1,1]), # 9
]

# And we print one of the vectors to show you how to get a specific vector
print(f'Digit 5 corresponds to the vector {x[5]}')

Digit 5 corresponds to the vector [1 0 1 1 0 1 1]


In [4]:
## Second we need to create ten perceptron with weights and biases
# You also need to figure out which weights and biases to use for each perceptron
# We've already created some of the perceptrons for you, but you need to create the rest
# While we're using integers for our weight you can use floating point numbers (real numbers) as well if you want

weights = [
    numpy.array([1,1,1,1,1,1,-1]), # 0
    numpy.array([-1,1,1,-1,-1,-1,-1]), # 1
    numpy.array([1,1,-1,1,1,-1,1]), # 2
    numpy.array([1,1,1,1,-1,-1,1]), # 3
    numpy.array([-1,1,1,-1,-1,1,1]), # 4
    numpy.array([1,-1,1,1,-1,1,1]), # 5
    numpy.array([1,-1,1,1,1,1,1]), # 6
    numpy.array([1,1,1,-1,-1,-1,-1]), # 7
    numpy.array([1,1,1,1,1,1,1]), # 8
    numpy.array([1,1,1,1,-1,1,1]), # 9

]

biases = [-sum(s) + 1 for s in x]
print(biases)

[-5, -1, -4, -4, -3, -4, -5, -2, -6, -5]


In [5]:
## Finally let's evaluate the output of the perceptrons.
# The line computing the output of each perceptron is not using bias, so you need to add that as well.
# If the perceptrons dont give the correct output then go back and edit the weights and biases in the
# previous cell (or check that the vectors are implemented correctly) until the outputs discriminate
# correctly between different digits. Remember to run a cell again (CONTROL + ENTER) if you update it.

debug = True # Set this to True if the predictions are wrong to get a more detailed output

for digit in range(10): # For each digit between 0 and 9 (range(n) gives a range (almost a list) of each number between 0 and n (excluding n)
    vector = x[digit] # Get the correct vector representation of the digit

    outputs = [] # Create an empty list to put the perceptrons' outputs in
    for w, b in zip(weights, biases): # For each weight and bias in the lists (zip takes two lists [x1,x2,...] [y1,y2,...] and makes a new list [(x1,y1),(x2,y2),...])

        # CHANGE THIS LINE TO ADD BIASES AS WELL
        output = w.dot(vector) + b  # Calculating the output of the perceptron with weight w and bias b

        outputs.append(output) # Adding the output to the list of outputs
    prediction = outputs.index(max(outputs)) # Get prediction by taking the index of the output value with maximum input

    print(f'Digit {digit} was predicted to be {prediction}') # This is an f-string with notation f'text {variable1} more text {variable2}'

    if debug: # If debug is True
        print(f'Outputs for all perceptrons for the digit {digit}: {outputs}')
        print() # Add a newline for formating

Digit 0 was predicted to be 0
Outputs for all perceptrons for the digit 0: [1, -3, -2, -2, -3, -2, -1, -2, 0, -1]

Digit 1 was predicted to be 1
Outputs for all perceptrons for the digit 1: [-3, 1, -4, -2, -1, -4, -5, 0, -4, -3]

Digit 2 was predicted to be 2
Outputs for all perceptrons for the digit 2: [-2, -4, 1, -1, -4, -3, -2, -3, -1, -2]

Digit 3 was predicted to be 3
Outputs for all perceptrons for the digit 3: [-2, -2, -1, 1, -2, -1, -2, -1, -1, 0]

Digit 4 was predicted to be 4
Outputs for all perceptrons for the digit 4: [-3, -1, -4, -2, 1, -2, -3, -2, -2, -1]

Digit 5 was predicted to be 5
Outputs for all perceptrons for the digit 5: [-2, -4, -3, -1, -2, 1, 0, -3, -1, 0]

Digit 6 was predicted to be 6
Outputs for all perceptrons for the digit 6: [-1, -5, -2, -2, -3, 0, 1, -4, 0, -1]

Digit 7 was predicted to be 7
Outputs for all perceptrons for the digit 7: [-2, 0, -3, -1, -2, -3, -4, 1, -3, -2]

Digit 8 was predicted to be 8
Outputs for all perceptrons for the digit 8: [0, -

### Task 1.2

Next you will train a classifier for the seven-segment display digits using PyTorch, much like you did in Exercise 0 with another dataset. The goal is to learn the weights and biases to correctly classify the digits of a seven-segment display. For this task you have the data needed via the first task above.

Create the network using the PyTorch model [torch.nn.Linear](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html), which you can find more information about in the [PyTorch documentation](https://pytorch.org/docs/stable/index.html). Whenever you encounter a new PyTorch function or class it's helpful to look it up in the documentation. If you wonder whether a particular machine learning feature exists in PyTorch you can also search in the documentation, or use the more brute force method of web search or asking an artificial intelligence for further information (see the discussion forum about AI in Canvas for further information).

In [6]:
## Data and labels
# You don't need to edit this code, we simply show you how, in this case, we create the data and labels we need

# A matrix (or vector) in PyTorch is usually represented by a Tensor
# Create a Tensor with our digit vectors
data = torch.Tensor(np.array(x)) # A Tensor can be created by simply giving it a nested numpy array/list of numbers as input
data = data.detach() # Since we won't be chaning the data during training we detach the Tensor from the computation graph

# Our labels will be the expected outputs of each perceptron for each digit
# This means we can't simply say "5" is our expected output, but rather that we want [0,0,0,0,0,1,0,0,0,0]
# This is called one-hot encoding where we have a vector where the value is one at the given index and zero everywhere else
# Since a matrix with one-hot representations of the corresponding index is simply an identity matrix, we use that as our labels
labels = torch.eye(10) # Get a matrix with one-hot representations of each digit in each row (an identity matrix)
labels = labels.detach() # Since we won't be changing the labels during training we detach the Tensor from the computation graph

NameError: name 'np' is not defined

In [ ]:
## The network
# We create a simple network from torch.nn.Linear

input_size = 7 # What is the size of the input vector to the network?
output_size = 10 # What is the size of the output vector of the network?

network1 = torch.nn.Linear(input_size, output_size) # Creating a single linear layer of a neural network with the given input and output sizes
network2 = torch.nn.Linear(input_size, output_size) # Creating a single linear layer of a neural network with the given input and output sizes

In [ ]:
## Training the network
# You don't need to edit this code, we have given you the training loop to train the network

epochs = 30 # How many epochs (complete runs of the data) to train for. Since our dataset is small 100 seems reasonable
loss_function = torch.nn.MSELoss() # What function to use to calculate the loss given the prediction and labels
optimizer = torch.optim.SGD(network1.parameters(), lr=1) # Function for updating the parameters of the network based on loss
learning_rate = 1 # How fast to optimize the network. Since our problem is quite small we can have a large learning rate, otherise 0.01 is usually standard

# Create a list to keep track of how the loss changes
losses = []

# For each epoch
for epoch in range(epochs):

    # Predict for each vector what digit they represent
    prediction = network1(data)

    # Calculate the loss of the prediction by comparing to the expected output
    loss = loss_function(prediction, labels)

    # Backpropogate the loss through the network to find the gradients of all parameters
    loss.backward()

    # Update the parameters along their gradients
    optimizer.step()

    # Clear stored gradient values
    optimizer.zero_grad()

    # Add the loss to the total epoch loss (item() turns a PyTorch scalar into a normal Python datatype)
    losses.append(loss.item())

    #Print the epoch and loss every 10 epochs
#    if epoch % 10 == 9:
#        print(f'Epoch {epoch+1} - Loss: {loss}')

# Plot the training loss per epoch
print("Network 1 output", network1(data))

#for name, param in network1.named_parameters():
#  print(name, param)

# Plot the training loss per epoch
plt.plot(range(1,epochs+1),losses)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()

## Training the network
# You don't need to edit this code, we have given you the training loop to train the network

epochs = 1000 # How many epochs (complete runs of the data) to train for. Since our dataset is small 100 seems reasonable
loss_function = torch.nn.MSELoss() # What function to use to calculate the loss given the prediction and labels
optimizer = torch.optim.SGD(network2.parameters(), lr=1) # Function for updating the parameters of the network based on loss
learning_rate = 1 # How fast to optimize the network. Since our problem is quite small we can have a large learning rate, otherise 0.01 is usually standard

# Create a list to keep track of how the loss changes
losses = []

# For each epoch
for epoch in range(epochs):

    # Predict for each vector what digit they represent
    prediction = network2(data)

    # Calculate the loss of the prediction by comparing to the expected output
    loss = loss_function(prediction, labels)

    # Backpropogate the loss through the network to find the gradients of all parameters
    loss.backward()

    # Update the parameters along their gradients
    optimizer.step()

    # Clear stored gradient values
    optimizer.zero_grad()

    # Add the loss to the total epoch loss (item() turns a PyTorch scalar into a normal Python datatype)
    losses.append(loss.item())

print(f"Network2 output {network2(data)}")

# Plot the training loss per epoch
plt.plot(range(1,epochs+1),losses)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()

prediction = network2(data).detach()
fig, ax0 = plt.subplots(1, 1)
digits = numpy.arange(10)  # len = 6
X, Y = numpy.meshgrid(digits, digits)
im = ax0.pcolormesh(X, Y, prediction, vmin=0.0, vmax=1.0)  # vmax=0.8
fig.colorbar(im, ax=ax0)
plt.show()

print(f"Labels {labels}")


### Task 1.3: Check the solution

Execute the cell below to see whether the network managed to learn to make the correct predictions.
Can you figure out what the learned weights and biases are, and how similar they are to your hardcoded solutions in the first task?

In [ ]:
## Testing the trained network
# You don't need to edit this code

with torch.no_grad(): # Since we're not training we don't want to calculate the gradients for this prediction
    prediction = network1(data) # Let's make one final prediction of the data

for digit in range(10):
    print(f'Digit {digit} was predicted to be {torch.argmax(prediction[digit])}') # argmax gets the index with the greatest value

## Task 2

While it is unnecessary to use machine learning to develop classifiers for a seven-segment display there are many other datasets for which it is difficult, if not impossible, to engineer such solutions without data-driven training/optimization of a model. For example, we have no generally useful and efficient method enabling us to write a program that can tell the difference between images of cats and dogs. However, we can easily gather many images of each, label the images, and use the resulting dataset to train a neural network that solves the task. A central problem in machine learning is how to define and train such a model to obtain maximum performance on new data which has not been used to optimize the model, referred to as the generalization of the model. Another important problem in machine learning is to minimize the computational cost of training and using such models.

Next you will extend the work started in Exercise 0 by developing a 2-layer neural network for classification of the MNIST dataset. MNIST consist of 70,000 grayscale images of handwritten digits that are 28x28 pixels each. Our goal is to train a 2-layer network that can recognize what digit an image represents. The subtasks are

* Implementation of a 2 layer NN (very similar to ex0)
* Training of this 2 layer NN (once again, very similar to ex0)
* Validation of the network during training (requires splitting the training set)
    - Save the model which performs the best on the validation data
* Graph the training loss vs validation loss
* You should obtain at least 85% accuracy on the test data (remember to load the best performing model before performing the accuracy test)

The code below loads the dataset (downloads it if necessary) and displays one of the images. You have to modify this code to complete this task.

In [ ]:
# Define the mini-batch size
batch_size = 1000

train_loader = DataLoader(parts[0], batch_size=batch_size, shuffle=False)
validate_loader = DataLoader(parts[1], batch_size=batch_size, shuffle=False)
print("Train", type(train_loader))

test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=False)

to_onehot = nn.Embedding(10, 10)
to_onehot.weight.data = torch.eye(10)

def plot_digit(data):
    data = data.view(28, 28)
    plt.imshow(data, cmap="gray")
    plt.show()

images, labels = next(iter(train_loader))
plot_digit(images[0])

### Task 2.1

Implement a 2-layer neural network using pytorch as well as a procedure for training and testing it. The training protocol should include both training and validation. Thus you need to split the training data into a training set (for which the error is backpropagated to update the parameters) and a validation set (which will not be used to directly update the model parameters, and instead be used to keep track of how good the model is at unseen data).

The weights of the model which performs the best on the validation data should be stored and then be used for the final check on the test set. Validation sets are often created by taking a fraction of the training data (often, but not always, around 20%) at random. In Pytorch you might want to use [random_split](https://pytorch.org/docs/stable/data.html#torch.utils.data.random_split) for this. Using random split would require you to edit the way the Dataloaders are created.

You are free to choose any optimizer and loss function. Just note that some loss functions require the labels to be 1-hot encoded. As you will not use convolutional layers for this exercise (will be introduced later in the course), the inputs need to be transformed to 1d tensors (see [view](https://pytorch.org/docs/stable/tensors.html?highlight=view#torch.Tensor.view)).

**GOAL:** You should evaluate the network from the epoch with best validation score (early stopping) on the test set aiming to reach at least 85% accuracy.

**Remember** to run all your code before grading so that the teacher doesn't have to wait around for long training runs. Plot the training and validation losses for each epoch.

*Hint:* Validation and Testing loops are very similar to training except they don't use backpropagation. Additionally testing should only be performed once, while validation should be performed continually to make sure training is proceeding as intended and to save the parameters of the best epoch.

*Hint:* Storing the best model is more difficult than just assigning it to a variable as this only means you have two variables referencing the same network instance in memory (not a copy of the best betwork and one containing the current network). Instead you ned to make a copy of the network which can be achived with [deepcopy](https://docs.python.org/3/library/copy.html). Other ways to store models include saving them as a file which can be done with [torch.save](https://pytorch.org/tutorials/beginner/saving_loading_models.html).

*Hint:* Everytime you train a network with random parameter initialization and random batches you get networks with different performance. Sometimes just running the training again can be enough to get a better result. However, if you do this too many times you run the risk of training (overfitting) on the test set.

In [ ]:
# Implement the network here using PyTorch
net = nn.Sequential(
    nn.Linear(28 * 28, 100),
    nn.Linear(100, 10),
).to(device)

def build_input(images):
  return images.view(-1, 28 * 28)

learning_rate = 1e-3
optimizer = torch.optim.RMSprop(net.parameters(), lr=learning_rate)

_m = nn.Sigmoid()
_loss_fn = nn.BCELoss()
def loss_fn(prediction, labels):
  return _loss_fn(_m(prediction), labels)

def to_onehot(tensor):
  return nn.functional.one_hot(tensor, num_classes=10)

epocs = 100
epocs_since_best = 0
prev_fraction = 0.0
best_fraction = 0.0
best_net = None
for epoch in range(epocs):
  # Training
  for batch_no, (images, labels) in zip(itertools.count(), iter(train_loader)):
    labels = to_onehot(labels).type(torch.Tensor).to(device)
    images = build_input(images).to(device)

    prediction = net(images)

    loss = loss_fn(prediction, labels)

    net.zero_grad()
    gradients = loss.backward()
    optimizer.step()

  # Validation
  correct = 0
  validation_size = 0
  for batch_no, (images, labels) in zip(itertools.count(), iter(validate_loader)):
    labels = labels.to(device)
    images = build_input(images).to(device)

    predictions = net(images)

    guesses = torch.argmax(predictions, dim=1)

    correct += torch.sum(guesses == labels)
    validation_size += labels.shape[0]

  epocs_since_best += 1
  correct_fraction = correct / validation_size
  if correct_fraction > best_fraction:
    direction = "best yet"
    best_net = copy.deepcopy(net)
    best_fraction = correct_fraction
    epocs_since_best = 0
  elif correct_fraction > prev_fraction:
    direction = "up"
  else:
    direction = "stagnant or down"
  prev_fraction = correct_fraction
  print(f"Epoch {epoch} not improving {epocs_since_best}. Total validation success {100 * correct_fraction:.1f}% {direction}")
  if epocs_since_best > 8:
      print("early break - no progress")
      break



In [ ]:
# Test
correct = 0
validation_size = 0
for batch_no, (images, labels) in zip(itertools.count(), iter(test_loader)):
  labels = labels.to(device)
  images = build_input(images).to(device)

  predictions = best_net(images)

  guesses = torch.argmax(predictions, dim=1)

  correct += torch.sum(guesses == labels)
  validation_size += labels.shape[0]

correct_fraction = correct / validation_size
print(f"Test. Total test success {100 * correct_fraction:.1f}%")


## Task 3

Implement a 2-layer neural network like above, but now without using a high-level machine learning library like pytorch. The network should function in the same way as the network in Task 2. The code block below contains code to change the training data from the DataLoader format to the "raw" numpy format. It also contains some comments to guide you in the right direction. The substasks involved are

* Implementation of a 2 layer NN using numpy
* Training and validation of the 2 layer NN
    - Once again, save the best performing model (can be done in memory)
* Graph the training vs validation loss
* At least 50% accuracy on the validation data (can be hard to get high accuracy)

**Note that the solution does not need to be fast/scalable, thus it is OK to develop a custom solution with, e.g., two explicit weight matrix variables representing the two layers.**

### The foward pass
This is easy. Recall that each differnt layer is calculated by the formula:
$$ y = g(\mathbf{W}*\mathbf{x} + b) $$
where $W$ is the weight matrix, $x$ the input, $b$ the bias and $g$ the non-linearity. For this exercise you are allowed to put $b = 0$ for simplicity when calculating the backwards pass.

### Backward pass
This can be tricky. In canvas there is lecture material which explains back propogation and all the maths behind it. It should be under *Modules > Artificial Neural Networks (ANN) - Part 1 > Lecture: Backpropagation derivation.mp4*. This, the supplementary material for the lecture, together with the course book should be enough material for you to be able to implement the training algorithm.

### Weight update
Once you have calculated the gradient of both weight matrixes, this is updated by:
$$ W_i = W_i - \gamma \dfrac{dL}{dW_i} $$
where $\gamma$ is the step size, or learning rate.

***Remember*** to run all your code before grading so the teacher doesn't have to wait around for long training runs.

In [ ]:
import sys
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = 'trace'
import time

import itertools
import matplotlib.pyplot as plt
from functools import partial

import jax
import jax.numpy as jnp
import torch
from torch.utils.data import DataLoader
from torchvision.transforms import v2
from torchvision import datasets, transforms


print("devices", jax.devices())

train_batch_size = 32
batch_size = 64
epochs = 50  # Set the number of epochs to train for
D_in = 28*28   # Input size, images are 28x28 = 784 element vectors
D_out = 10   # Output size, 10 digit classes
H1 = 256     # Hidden layer size
gamma = 1e-3 / batch_size # Learning rate

mnist_train = datasets.MNIST("./", train=True, download=True, transform=v2.Compose([v2.ToImage(),
                                                                                    v2.ToDtype(torch.float32, scale=True)]))
parts = torch.utils.data.random_split(mnist_train, [0.8, 0.2])
mnist_test = datasets.MNIST("./", train=False, download=True, transform=v2.Compose([v2.ToImage(),
                                                                                    v2.ToDtype(torch.float32, scale=True)]))

train_loader = DataLoader(parts[0], batch_size=train_batch_size, shuffle=False, num_workers=0)
validate_loader = DataLoader(parts[1], batch_size=batch_size, shuffle=False, num_workers=0)
test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=False, num_workers=0)

# Define network with one hidden layer, random initial weights
rnd = jax.random.PRNGKey(42)
rnd, rw1, rw2, rb1, rb2 = jax.random.split(rnd, 5)
w1 = jax.random.normal(rw1, shape=(H1, D_in))
w2 = jax.random.normal(rw2, shape=(D_out, H1))

b1=jax.random.normal(rb1, shape=(H1,))
b2=jax.random.normal(rb2, shape=(D_out,))

# Training iterations
@partial(jax.jit, inline=True)
def build_input(images: jax.Array) -> jax.Array:
  return images.view(-1, 28 * 28)

@partial(jax.jit, inline=True)
def to_onehot(labels: jax.Array) -> jax.Array:
  return jnp.eye(10)[labels]

EPSILON = 1e-6

@partial(jax.jit, inline=True)
def softmax(y_pred):
  y_pred = jnp.array(y_pred, dtype=jnp.float32)
  y_max = jnp.max(y_pred, axis=1)
  z = jnp.subtract(y_pred.T, y_max).T  # for numerical stability
  z_exp = jnp.exp(z)
  z_exp_sum = jnp.sum(z_exp, axis=1)
  return (z_exp.T / z_exp_sum).T


@partial(jax.jit, inline=True)
def binary_cross_entropy(y_pred, y_true, axis=0):
  """ Computes binary cross entropy between two distributions.
  Input: y_pred: iterable of N non-negative values
         y_true: iterable of N non-negative values
  Returns: scalar
  """

  # Force to proper probability mass function.
  y_true = jnp.array(y_true, dtype=jnp.float32)
  y_pred = jnp.array(y_pred, dtype=jnp.float32)

  # Ignore zero and negative 'y_pred' elements.
  y_pred_when_one = jnp.where(y_pred > 0,
                              y_pred,
                              jnp.exp(-1))  # y_true = 1, y_pred = 0 should give -1

  y_pred_when_zero = jnp.where((1 - y_pred) > 0,
                               1 - y_pred,
                               jnp.exp(-1))  # y_true = 0 y_pred = 1 should give -1

  multiply_01 = jnp.log(jnp.where(y_true == 1,
                                  y_pred_when_one,
                                  y_pred_when_zero))

  if False:
    # d/dx lg(x) = 1/x
    # -(y_true / y_pred + g'(y_pred) * (1 - y_true) / (1 - y_pred)) where g(x) = 1 - y_pred
    # rewrite to common denominator, make sure divisor does not get very close to 0
    # TODO softmax?
    loss_d = jnp.divide(y_pred - y_true,
                        jnp.multiply(y_pred - 1, y_pred))
  else:
    # Categorical Cross-Entropy Loss
    # https://towardsdatascience.com/derivative-of-the-softmax-function-and-the-categorical-cross-entropy-loss-ffceefc081d1
    loss_d = y_pred - y_true  # Note: y_pred is limited by softmax

  return -multiply_01.mean(axis=1), loss_d

def test_cross_entropy():
    tbce = binary_cross_entropy([[0.5, 0.5, 0, 1, 0, 1]],
                                [[0,   1,   0, 0, 1, 1]])
    print("test binary_cross_entropy", tbce)
    y_pred_sm_ = softmax([[0.5, 0.5, 0, 1, 0, 1]])
    print("y_pred softmax", y_pred_sm_)
    tbce = binary_cross_entropy(y_pred_sm_,
                                [[0,   1,   0, 0, 1, 1]])
    print("test binary_cross_entropy softmax", tbce)

# test_cross_entropy()

@partial(jax.jit, inline=True)
def loss_fn(y_pred, y):
  if False:
    # predictions > 1 are interpreted as 1, predictions < 0 are interpreted as 0
    err=np.subtract(y_pred.clip(0, 1), y)
    loss = np.sum(np.power(err, 2), axis=1)
    loss_d = 2*err
  else:
    y_pred_ = softmax(y_pred)
    loss, loss_d = binary_cross_entropy(y_pred_, y)
  return loss, loss_d


@partial(jax.jit, inline=True)
def collect(h1_: jax.Array, g: jax.Array, x: jax.Array, w2: jax.Array):
    #layer2
    #activater function g=g cause we don't have an activater function
    #lamda=0 no regulization then
    b2_d = g.reshape((D_out,))
    w2_d = jnp.matmul(g, h1_.T)
    g = jnp.matmul(w2.T, g)
    #layer1
    b1_d = g.reshape((H1,))
    w1_d = jnp.matmul(g, x.T)

    return (w1_d, b1_d), (w2_d, b2_d)

vmap_collect = jax.jit(jax.vmap(collect, in_axes=(0, 0, 0, None)))

@partial(jax.jit)
def train_batch(inputs: jax.Array, labels: jax.Array, w1: jax.Array, b1: jax.Array, w2: jax.Array, b2: jax.Array):
    # iterate through the mini-batch and perform forward pass and backward pass
    x = inputs.reshape((-1, D_in))
    y = to_onehot(labels)    # 1-hot encoding

    # Forward pass
    h1 = jnp.matmul(x, w1.T) + b1
    h2 = jnp.matmul(h1,w2.T) + b2
    #h_relu = jnp.maximum(0,h1) # x * (x > 0)
    y_pred = h2

    # Compute loss function, squared error
    loss, loss_d = loss_fn(y_pred, y)
    total_training_loss = loss.sum()

    # Compute gradients of square-error loss with respect to w1 and w2 using backpropagation
    #derivate of loss function


    der = vmap_collect(h1.reshape(-1, H1, 1),
                                                              loss_d.reshape(-1, D_out, 1),
                                                              x.reshape(-1, D_in, 1),
                                                              w2)
    (w1_da, b1_da), (w2_da, b2_da) = jax.tree_util.tree_map(lambda arr: jnp.sum(arr, axis=0), der)

    # Return update to accumulators
    return (total_training_loss,
            (w1 - gamma * w1_da, b1 - gamma * b1_da),
            (w2 - gamma * w2_da, b2 - gamma * b2_da))

@partial(jax.jit)
def validate_batch(x: jax.Array, w1: jax.Array, h1: jax.Array, w2: jax.Array, h2: jax.Array, labels: jax.Array):
    h1 = jnp.matmul(x, w1.T) + b1
    h2 = jnp.matmul(h1, w2.T) + b2
    predictions = h2

    guesses = jnp.argmax(predictions, axis=1)
    correct = jnp.sum(guesses == labels)
    loss, _ = loss_fn(predictions, to_onehot(labels))
    return correct, loss

prev_wall = time.perf_counter()
jit_train_batch = jax.jit(train_batch)
jit_validate_batch = jax.jit(validate_batch)
now_wall = time.perf_counter()
print(f"JAX compile time {now_wall - prev_wall:.3f}")
prev_wall = now_wall

print("big bang")
# Train for a number of epochs
plot_data = []
for epoch in range(epochs):
    # Training by looping over training set
    total_trainings = 0
    total_training_loss = 0.0
    for batch_no, (inputs, labels) in zip(itertools.count(), iter(train_loader)):
        inputs_ = inputs.numpy()  #x
        labels_ = labels.numpy()  #y

        # Compute loss function, squared error
        loss, (w1, b1), (w2, b2) = jit_train_batch(inputs_, labels_, w1, b1, w2, b2)

#        print(f"Epoch {epoch} batch {batch_no} loss {batch_training_loss / batch_size:.3f}")

        batch_trainings = labels.shape[0]
        batch_training_loss = loss.sum()
        total_trainings += batch_trainings
        total_training_loss += batch_training_loss

    # Validate the model
    total_validations = 0
    correct = 0
    total_validation_loss = 0.0
    for batch_no, (images, labels) in zip(itertools.count(), iter(validate_loader)):
      labels = labels.numpy()
      images = images.numpy()

      x = images.reshape((-1, D_in))

      batch_correct, batch_loss = jit_validate_batch(x, w1, b1, w2, b2, labels)
      correct += batch_correct
      total_validations += labels.shape[0]
      total_validation_loss += batch_loss.sum()

    correct_fraction = correct / total_validations
    validation_loss_mean = total_validation_loss / total_validations
    training_loss_mean = total_training_loss / total_trainings
    batch_training_loss_mean = batch_training_loss / batch_trainings  # last batch
    now_wall = time.perf_counter()
    print(f"Epoch {epoch} {now_wall - prev_wall:.3f}s Training loss {training_loss_mean:.3f}, last batch {batch_training_loss_mean:.3f} "
          f"Validation loss {validation_loss_mean:.3f} "
          f"success {100 * correct_fraction:.1f}%")
    prev_wall = now_wall
# Plot training and validation loss
    plot_data.append((training_loss_mean, batch_training_loss_mean, validation_loss_mean, correct_fraction))
    

In [ ]:
plot_data_ = numpy.array(plot_data)
epochs_ = plot_data_.shape[0]  # actual epochs recorded

# Plot the training loss per epoch
fig, ax = plt.subplots()
ttl = ax.plot(range(1, epochs_ + 1), plot_data_[:, 0:3])
ax.legend(["total training loss", "last batch training loss", "validation loss"])
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()

plt.plot(range(1, epochs_ + 1), plot_data_[:, 3])
plt.xlabel('Epochs')
plt.ylabel('Validation Correctness')
plt.show()



In [ ]:
# Calculate accuracy on one test batch

total = 0
correct = 0
for batch_no, (images, labels) in zip(itertools.count(), iter(test_loader)):
  labels = jnp.array(labels.numpy())
  images = jnp.array(images.numpy())

  x = images.reshape((-1, D_in))

  h1 = jnp.matmul(x, w1.T) + b1
  h2 = jnp.matmul(h1, w2.T) + b2
  predictions = h2

  guesses = jnp.argmax(predictions, axis=1)

  correct += jnp.sum(guesses == labels)
  total += labels.shape[0]

correct_fraction = correct / total
print(f"Test. Total test success {100 * correct_fraction:.1f}%")


## Optional programming challenge
You have now implemented a 2-layer neural network from scratch! Use this new gained knowledge to create an implementation where we can create and train a network with an arbitrary number of layers and units in each layer. This means that we should be able to specify the structure of the network and then train it using backpropagation.

## End

You have now reached the end of ANN1. When you have completed and understood the task above please make sure that all results inluding plots have been computed and then schedule a meeting with a teacher. The teacher will then assess orally that you (the lab group) has completed the exercise and that you understand its essental elements.